In [1]:
import os
import time
import cv2
import matplotlib.pyplot as plt
import numpy as np
import png
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from PIL import Image
from colormap.colors import Color, hex2rgb
from sklearn.metrics import average_precision_score as ap_score
from torch.utils.data import DataLoader
from torchvision import datasets, models, transforms
from tqdm import tqdm
from torch.utils.data.dataset import Dataset

In [2]:
"""
Helper functions.
"""
def save_label(label, path):
    '''
    Function for ploting labels.
    '''
    colormap = [
        '#000000',
        '#0080FF',
        '#80FF80',
        '#FF8000',
        '#FF0000',
    ]
    assert(np.max(label)<len(colormap))
    colors = [hex2rgb(color, normalise=False) for color in colormap]
    w = png.Writer(label.shape[1], label.shape[0], palette=colors, bitdepth=4)
    with open(path, 'wb') as f:
        w.write(f, label)

def train(trainloader, net, criterion, optimizer, device, epoch):
    '''
    Function for training.
    '''
    start = time.time()
    running_loss = 0.0
    net = net.train()
    for images, labels in tqdm(trainloader):
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        output = net(images)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        running_loss = loss.item()
    end = time.time()
    print('[epoch %d] loss: %.3f elapsed time %.3f' %
          (epoch, running_loss, end-start))

def test(testloader, net, criterion, device):
    '''
    Function for testing.
    '''
    losses = 0.
    cnt = 0
    with torch.no_grad():
        net = net.eval()
        for images, labels in tqdm(testloader):
            images = images.to(device)
            labels = labels.to(device)
            output = net(images)
            loss = criterion(output, labels)
            losses += loss.item()
            cnt += 1
    print(losses / cnt)
    return (losses/cnt)

def cal_AP(testloader, net, criterion, device):
    '''
    Calculate Average Precision
    '''
    losses = 0.
    cnt = 0
    with torch.no_grad():
        net = net.eval()
        preds = [[] for _ in range(5)]
        heatmaps = [[] for _ in range(5)]
        for images, labels in tqdm(testloader):
            images = images.to(device)
            labels = labels.to(device)
            output = net(images).cpu().numpy()
            for c in range(5):
                preds[c].append(output[:, c].reshape(-1))
                heatmaps[c].append(labels[:, c].cpu().numpy().reshape(-1))

        aps = []
        for c in range(5):
            preds[c] = np.concatenate(preds[c])
            heatmaps[c] = np.concatenate(heatmaps[c])
            if heatmaps[c].max() == 0:
                ap = float('nan')
            else:
                ap = ap_score(heatmaps[c], preds[c])
                aps.append(ap)
            print("AP = {}".format(ap))

    # print(losses / cnt)
    return None

def get_result(testloader, net, device, folder='./part3/output_train'):
    result = []
    cnt = 1
    with torch.no_grad():
        net = net.eval()
        cnt = 0
        for images, labels in tqdm(testloader):
            images = images.to(device)
            labels = labels.to(device)
            output = net(images)[0].cpu().numpy()
            c, h, w = output.shape
            assert(c == N_CLASS)
            y = np.zeros((h,w)).astype('uint8')
            for i in range(N_CLASS):
                mask = output[i]>0.5
                y[mask] = i
            gt = labels.cpu().data.numpy().squeeze(0).astype('uint8')
            save_label(y, './{}/y{}.png'.format(folder, cnt))
            save_label(gt, './{}/gt{}.png'.format(folder, cnt))
            plt.imsave(
                './{}/x{}.png'.format(folder, cnt),
                ((images[0].cpu().data.numpy()+1)*128).astype(np.uint8).transpose(1,2,0))

            cnt += 1

In [3]:
"""
Dataset.
"""
class FacadeDataset(Dataset):
    def __init__(self, flag, dataDir='./part3/starter_set/', data_range=(0, 8), n_class=5, onehot=False):
        self.onehot = onehot
        assert(flag in ['train', 'eval', 'test', 'test_dev', 'kaggle'])
        print("load "+ flag+" dataset start")
        print("    from: %s" % dataDir)
        print("    range: [%d, %d)" % (data_range[0], data_range[1]))
        self.dataset = []
        for i in range(data_range[0], data_range[1]):
            img = Image.open(os.path.join(dataDir,flag,'eecs442_%04d.jpg' % i))

            pngreader = png.Reader(filename=os.path.join(dataDir,flag,'eecs442_%04d.png' % i))
            w,h,row,info = pngreader.read()
            label = np.array(list(row)).astype('uint8')

            # Normalize input image
            img = np.asarray(img).astype("f").transpose(2, 0, 1)/128.0-1.0
            # Convert to n_class-dimensional onehot matrix
            label_ = np.asarray(label)
            label = np.zeros((n_class, img.shape[1], img.shape[2])).astype("i")
            for j in range(n_class):
                label[j, :] = label_ == j
            self.dataset.append((img, label))
        print("load dataset done")

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, index):
        img, label = self.dataset[index]
        label = torch.FloatTensor(label)
        if not self.onehot:
            label = torch.argmax(label, dim=0)
        else:
            label = label.long()

        return torch.FloatTensor(img), torch.LongTensor(label)

In [4]:
"""
DataLoaders.
"""
# TODO change data_range to include all train/evaluation/test data.
# TODO adjust batch_size.
train_data = FacadeDataset(
    flag='train',
    data_range=(0,20),
    onehot=False,
)
train_loader = DataLoader(train_data, batch_size=1)

test_data = FacadeDataset(
    flag='test_dev',
    data_range=(0,114),
    onehot=False
)
test_loader = DataLoader(test_data, batch_size=1)

ap_data = FacadeDataset(
    flag='test_dev',
    data_range=(0,114),
    onehot=True
)
ap_loader = DataLoader(ap_data, batch_size=1)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

load train dataset start
    from: ./part3/starter_set/
    range: [0, 20)
load dataset done
load test_dev dataset start
    from: ./part3/starter_set/
    range: [0, 114)
load dataset done
load test_dev dataset start
    from: ./part3/starter_set/
    range: [0, 114)
load dataset done


In [5]:
"""
CNN model.
"""
N_CLASS=5

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.n_class = N_CLASS
        self.layers = nn.Sequential(
            #########################################
            ###        TODO: Add more layers      ###
            #########################################
            nn.Conv2d(3, self.n_class, 1, padding=0),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        x = self.layers(x)
        return x

In [ ]:
"""
Main function.
"""
# init model
name = 'starter_net'
net = Net().to(device)
criterion = nn.CrossEntropyLoss() #TODO decide loss
optimizer = torch.optim.Adam(
    net.parameters(),
    1e-3,
    weight_decay=1e-5
)
n_epochs = 1   #TODO decide epochs

# train model
print('\nStart training')
for epoch in range(n_epochs):
    print('-----------------Epoch = %d-----------------' % (epoch+1))
    train(
        train_loader,
        net,
        criterion,
        optimizer,
        device,
        epoch+1
    )
    # TODO create your evaluation set, load the evaluation set and test on evaluation set
    evaluation_loader = train_loader
    test(
        evaluation_loader,
        net,
        criterion,
        device
    )

# test trained model
print('\nFinished Training, Testing on test set')
test(
    test_loader,
    net,
    criterion,
    device
)
print('\nGenerating Unlabeled Result')
result = get_result(
    test_loader,
    net,
    device,
    folder='./part3/output_test'
)

# save trained model
torch.save(
    net.state_dict(),
    './part3/models/model_{}.pth'.format(name)
)

# calculate average precision
cal_AP(ap_loader, net, criterion, device)


Start training
-----------------Epoch = 1-----------------


100%|███████████████████████████████████████████████████████| 20/20 [00:00<00:00, 60.97it/s]


[epoch 1] loss: 1.704 elapsed time 0.331


100%|███████████████████████████████████████████████████████| 20/20 [00:00<00:00, 80.64it/s]


1.6431577146053313

Finished Training, Testing on test set


100%|█████████████████████████████████████████████████████| 114/114 [00:01<00:00, 88.37it/s]


1.6071406979309886

Generating Unlabeled Result


 91%|████████████████████████████████████████████████▎    | 104/114 [00:14<00:01,  7.58it/s]